In [2]:
import numpy as np
from scipy.stats import norm, beta, uniform, truncnorm
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline

N=1
T=10
# a=0.5
# b=1.0
left = 0.0
stdev = 0.1
mean = 0.5
right = 1.0

a, b = (left - mean) / stdev, (right - mean) / stdev

samples = uniform.rvs(loc=0.0, scale=1.0, size=(N, T))
# samples = beta.rvs(a=a, b=b, size=(N, T))
# samples = truncnorm.rvs(a=a, b=b, loc=mean, scale=stdev, size=(N, T))
ord_stats = np.sort(samples, axis=0)

dist_mean = uniform.mean(loc=0.0, scale=1.0)
# dist_mean = beta.mean(a=a, b=b)
# dist_mean = truncnorm.mean(a=a, b=b, loc=mean, scale=stdev)

sample_mean = np.mean(samples)
print "minimum value of sample: ", np.min(samples)
print "maximum value of sample: ", np.max(samples)
print "distribution mean: ", dist_mean
print "sample mean (all samples): ", sample_mean
print "sample mean of a random trial: ", np.mean(samples[:, np.random.randint(low=0, high=T)])

minimum value of sample:  0.107802404384
maximum value of sample:  0.999607351328
distribution mean:  0.5
sample mean (all samples):  0.658286169728
sample mean of a random trial:  0.97781253223


/home/nkumar/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
## delta for N =  1
delta = 0.949494949495  # took this value from delta_values.csv

u_limit = np.array([beta.ppf(q=(1+delta)/2.0, a=i, b=N-i+1) for i in range(1, N+1)])
l_limit = np.array([beta.ppf(q=(1-delta)/2.0, a=i, b=N-i+1) for i in range(1, N+1)])

print "upper cdf: ", u_limit
print "lower cdf: ", l_limit

cdf_values = []
for t in xrange(T):
#     cdf_values.append(uniform.cdf(x=ord_stats[:, t], loc=0.0, scale=1.0))
#     cdf_values.append(beta.cdf(x=ord_stats[:, t], a=a, b=b))
    cdf_values.append(truncnorm.cdf(x=ord_stats[:, t], a=a, b=b, loc=mean, scale=stdev))
    
cdf_values = np.array(cdf_values)
#print "Max cdf value: ", max(cdf_values)
#print "Min cdf_value: ", min(cdf_values)

# plt.plot([1]*T, cdf_values, label='Sample CDF')
# plt.plot([1], u_limit, label='Upper CDF', linewidth=5.0)
# plt.plot([1], l_limit, label='Lower CDF', linewidth=5.0)
# # plt.legend(loc='best')

# # CDF violations
# u_violation = l_violation = 0.0

# for t in xrange(T):
#     if cdf_values[t] > u_limit[0]:
#         u_violation += 1
#     if cdf_values[t] < l_limit[0]:
#         l_violation += 1
        
# print "upper cdf violation: ", u_violation/T * 100.0
# print "upper cdf violation: ", l_violation/T * 100.0

upper cdf:  [ 0.97474747]
lower cdf:  [ 0.02525253]


In [4]:
def mean_integral_v1(cdf_values, ord_stats):
    cdf_values = np.hstack((0.0, cdf_values))
    samples = np.hstack((0.0, ord_stats))
    
    Fx = cdf_values[:-1] + np.diff(cdf_values)*0.5
    dx = np.diff(samples)
    print "printing dx"+str(dx)
    print "printing Fx"+str(1-Fx)
    mean = np.sum(np.multiply((1-Fx), dx))
    return mean

In [5]:
upper_mean = np.zeros(shape=T)
lower_mean = np.zeros(shape=T)

u_violation = l_violation = 0.0

for t in xrange(T):
    upper_mean[t] = mean_integral_v1(l_limit, ord_stats[:, t])
    lower_mean[t] = mean_integral_v1(u_limit, ord_stats[:, t])
    if upper_mean[t] < dist_mean:
        u_violation += 1
    if lower_mean[t] > dist_mean:
        l_violation += 1
        
print "upper mean violation percentage: ", u_violation/T * 100.0
print "lower mean violation percentage: ", l_violation/T * 100.0

printing dx[ 0.98602168]
printing Fx[ 0.98737374]
printing dx[ 0.98602168]
printing Fx[ 0.51262626]
printing dx[ 0.64399206]
printing Fx[ 0.98737374]
printing dx[ 0.64399206]
printing Fx[ 0.51262626]
printing dx[ 0.50039917]
printing Fx[ 0.98737374]
printing dx[ 0.50039917]
printing Fx[ 0.51262626]
printing dx[ 0.62697537]
printing Fx[ 0.98737374]
printing dx[ 0.62697537]
printing Fx[ 0.51262626]
printing dx[ 0.38517708]
printing Fx[ 0.98737374]
printing dx[ 0.38517708]
printing Fx[ 0.51262626]
printing dx[ 0.97781253]
printing Fx[ 0.98737374]
printing dx[ 0.97781253]
printing Fx[ 0.51262626]
printing dx[ 0.66012906]
printing Fx[ 0.98737374]
printing dx[ 0.66012906]
printing Fx[ 0.51262626]
printing dx[ 0.1078024]
printing Fx[ 0.98737374]
printing dx[ 0.1078024]
printing Fx[ 0.51262626]
printing dx[ 0.99960735]
printing Fx[ 0.98737374]
printing dx[ 0.99960735]
printing Fx[ 0.51262626]
printing dx[ 0.69494498]
printing Fx[ 0.98737374]
printing dx[ 0.69494498]
printing Fx[ 0.51262626]
up

In [6]:
print "percentile of mean: ", beta.cdf(beta.mean(a=0.5, b=1), a=1.0, b=1.0)

percentile of mean:  0.333333333333


In [116]:
def mean_integral_v1(cdf_values, ord_stats):
    cdf_values = np.hstack((0.0, cdf_values, 1.0))
    cdf_values = 1.0 - cdf_values
    
#     print "cdf_values: ", cdf_values
    
    samples = np.hstack((0.0, ord_stats, 1.0))
#     print "samples: ", samples
    
    Fx = cdf_values[:-1] + np.diff(cdf_values)*0.5
#     print "Fx values: ", Fx
    
    dx = np.diff(samples)
#     print "dx values: ", dx
    
    mean = np.sum(np.multiply(Fx, dx))
    return mean



order_stats = np.array([1.0])
mean_lower_cdf = np.array([0.97474747])
mean_upper_cdf = np.array([0.02525253])

print("lower bound on mean",mean_integral_v1(mean_lower_cdf, order_stats))
print("upper bound on mean",mean_integral_v1(mean_upper_cdf, order_stats))


('lower bound on mean', 0.51262626499999997)
('upper bound on mean', 0.98737373500000003)


In [118]:
from scipy.stats import beta

N=1
T=100000
a=0.5
b=0.5

u_limit = 0.97474747
l_limit = 0.02525253
violation = 0.0

dist_mean = beta.mean(a=a, b=b)
print "mean of the distribution: ", dist_mean
print "cdf of the mean of the distribution: ", beta.cdf(dist_mean, a=0.5, b=1.0)

samples = beta.rvs(a=0.5, b=1.0, size=(N, T))
sample_cdf_values = np.zeros(T, dtype='float')


# for t in xrange(T):
#     sample_cdf = beta.cdf(samples[:, t], a=a, b=b)
#     if sample_cdf > u_limit or sample_cdf < l_limit:
#         violation += 1.0
# print "cdf violations: ", violation/T * 100.0
        
sorted_z1 = np.sort(samples, axis=1)
percentile = int(0.95*T)

t = sorted_z1[:, percentile]
cdf_t = beta.cdf(t, a=a,b=b)

print "Value of t in the 95th percentile: ", t
print "cdf value of t: ", cdf_t

def mean_integral(cdf_values, ord_stats):
    cdf_values = np.hstack((0.0, cdf_values, 1.0))
    cdf_values = 1.0 - cdf_values
    samples = np.hstack((0.0, ord_stats, 1.0))
    Fx = cdf_values[:-1] + np.diff(cdf_values)*0.5
    dx = np.diff(samples)
    
    mean = np.sum(np.multiply(Fx, dx))
    return mean

order_stats = t
upper_cdf = np.array([u_limit])
lower_cdf = np.array([l_limit])

est_lower_mean = mean_integral(upper_cdf, order_stats)
# est_upper_mean = mean_integral(lower_cdf, order_stats)
print "Lower mean: ", est_lower_mean
# print "Upper mean: ", est_upper_mean


mean of the distribution:  0.5
cdf of the mean of the distribution:  0.707106781187
Value of t in the 95th percentile:  [ 0.90347958]
cdf value of t:  [ 0.79888827]
Lower mean:  0.464366054214


In [102]:
from scipy.stats import uniform

N=1
T=100000
loc=0.0
scale=1.0

u_limit = 0.97474747
l_limit = 0.02525253
violation = 0.0

dist_mean = uniform.mean(loc=loc, scale=scale)
print "cdf of the mean of the distribution: ", uniform.cdf(dist_mean, loc=loc, scale=scale)

samples = uniform.rvs(loc=loc, scale=scale, size=(N, T))
sample_cdf_values = np.zeros(T, dtype='float')


# for t in xrange(T):
#     sample_cdf = uniform.cdf(samples[:, t], loc=loc, scale=scale)
#     if sample_cdf > u_limit or sample_cdf < l_limit:
#         violation += 1.0
# print "cdf violations: ", violation/T * 100.0
        
sorted_z1 = np.sort(samples, axis=1)
percentile = int(0.95*T)

t = sorted_z1[:, percentile]
cdf_t = uniform.cdf(t, loc=loc, scale=scale)

print "Value of t in the 95th percentile: ", t
print "cdf value of t: ", cdf_t

def mean_integral(cdf_values, ord_stats):
    cdf_values = np.hstack((0.0, cdf_values, 1.0))
    cdf_values = 1.0 - cdf_values
    samples = np.hstack((0.0, ord_stats, 1.0))
    Fx = cdf_values[:-1] + np.diff(cdf_values)*0.5
    dx = np.diff(samples)
    
    mean = np.sum(np.multiply(Fx, dx))
    return mean

order_stats = np.array([t])
upper_cdf = np.array([u_limit])
lower_cdf = np.array([l_limit])

est_lower_mean = mean_integral(upper_cdf, order_stats)
est_upper_mean = mean_integral(lower_cdf, order_stats)
print "Lower mean: ", est_lower_mean
print "Upper mean: ", est_upper_mean

cdf of the mean of the distribution:  0.5
Value of t in the 95th percentile:  [ 0.94952955]
cdf value of t:  [ 0.94952955]


In [114]:
print "cdf value: ", beta.cdf(0.66, a=0.5, b=1.0)

cdf value:  0.812403840464
